In [81]:
import pandas as pd
import geopandas
from shapely import wkt

def read_toy_dataset(path, column_names):
    traj_df = pd.read_csv(filepath_or_buffer=path, compression='gzip', header=None, sep='\t', names=column_names)
    traj_df['ts'] = pd.to_datetime(traj_df['ts'], unit='ms')
    traj_df['geom'] = traj_df['geom'].apply(wkt.loads)
    traj_df = geopandas.GeoDataFrame(traj_df, geometry='geom')
    return traj_df

path = './toy_traj.csv.gz'
columns = ['id', 'ts', 'geom']
tdf = read_toy_dataset(path, columns)
tdf.head()

,id,ts,geom
0,2415,2012-11-24 16:50:00,"POLYGON ((-920.700 152.100, -920.100 152.700, ..."
1,2415,2012-11-24 17:00:00,"POLYGON ((-920.700 152.100, -881.700 160.500, ..."
2,2415,2012-11-24 17:10:00,"POLYGON ((-920.700 152.100, -881.700 160.500, ..."
3,2415,2012-11-24 17:20:00,"POLYGON ((-920.700 152.100, -881.700 160.500, ..."
4,2415,2012-11-24 17:30:00,"POLYGON ((-920.700 152.100, -881.700 160.500, ..."


In [82]:
class Trajectory:
  """
  A trajectory is a list of time-geopoint pairs.
  """
  def __init__(self, traj_id, tgpairs):
    self.traj_id = traj_id
    self.tgpairs = tgpairs

  def __str__(self):
    #return f"{self.traj_id}" # replace as needed
    return f"Trajectory {self.traj_id}: {len(self.tgpairs)} points"

In [83]:
import numpy as np
from intervaltree import Interval, IntervalTree
from shapely.geometry import box

class GIT:
    """ Grid Index for Trajectories (GIT)
    A grid index for trajectories that supports the following operations:
    - Insertion of a trajectory
    - Deletion of a trajectory
    - Spatial window query
    - Temporal window query
    - Spatiotemporal window query
    :param sf_xmin: float
    :param sf_xmax: float
    :param sf_ymin: float
    :param sf_ymax: float
    :param delta_x: float
    :param delta_y: float
    """
    def __init__(self, sf_xmin, sf_xmax, sf_ymin, sf_ymax, delta_x, delta_y):
        self.sf_xmin = sf_xmin
        self.sf_xmax = sf_xmax
        self.sf_ymin = sf_ymin
        self.sf_ymax = sf_ymax
        self.delta_x = delta_x
        self.delta_y = delta_y
        self.nx = int(np.ceil((sf_xmax - sf_xmin) / delta_x))
        self.ny = int(np.ceil((sf_ymax - sf_ymin) / delta_y))
        self.grid = [[IntervalTree() for _ in range(self.ny)] for _ in range(self.nx)]

    def _get_intersecting_cells(self, geometry):
        """
        Returns a list of cell coordinates (x, y) that intersect with the given geometry.
        :param geometry: shapely.geometry
        :return: list of tuples (x, y)
        """
        cells = []
        for i in range(self.nx):
            for j in range(self.ny):
                cell_bbox = box(self.sf_xmin + i * self.delta_x, self.sf_ymin + j * self.delta_y,
                                self.sf_xmin + (i + 1) * self.delta_x, self.sf_ymin + (j + 1) * self.delta_y)
                if geometry.intersects(cell_bbox):
                    cells.append((i, j))
        return cells


    def insert(self, trajectory):
        """
        Inserts a trajectory into the grid index.
        :param trajectory: Trajectory
        """
        for i in range(len(trajectory.tgpairs) - 1):
            tgi_start, tgi_end = trajectory.tgpairs[i], trajectory.tgpairs[i + 1]
            ti_start, gi_start = tgi_start
            ti_end, gi_end = tgi_end
            intersecting_cells = self._get_intersecting_cells(gi_start.union(gi_end))

            for cell_coords in intersecting_cells:
                x, y = cell_coords
                self.grid[x][y].add(Interval(ti_start, ti_end, trajectory.traj_id))

    def delete(self, trajectory_id):
        """
        Deletes a trajectory from the grid index.
        :param trajectory_id: int
        """
        for i in range(self.nx):
            for j in range(self.ny):
                intervals_to_remove = [interval for interval in self.grid[i][j] if interval.data == trajectory_id]
                for interval in intervals_to_remove:
                    self.grid[i][j].remove(interval)

    def spatial_window_query(self, x1, y1, x2, y2):
        """
        Returns a set of trajectory ids that intersect with the given spatial window.
        :param x1: float
        :param y1: float
        :param x2: float
        :param y2: float
        :return: set of int
        """
        query_bbox = box(x1, y1, x2, y2)
        result = set()

        for i in range(self.nx):
            for j in range(self.ny):
                cell_bbox = box(self.sf_xmin + i * self.delta_x, self.sf_ymin + j * self.delta_y,
                                self.sf_xmin + (i + 1) * self.delta_x, self.sf_ymin + (j + 1) * self.delta_y)
                if query_bbox.intersects(cell_bbox):
                    for interval in self.grid[i][j]:
                        result.add(interval.data)
        return result

    def temporal_window_query(self, t1, t2):
        """
        Returns a set of trajectory ids that intersect with the given temporal window.
        :param t1: timestamp
        :param t2: timestamp
        :return: set of int
        """
        result = set()
        for i in range(self.nx):
            for j in range(self.ny):
                intervals = self.grid[i][j][t1:t2]
                for interval in intervals:
                    result.add(interval.data)
        return result

    def spatiotemporal_window_query(self, x1, y1, x2, y2, t1, t2):
        """ 
        Returns a set of trajectory ids that intersect with the given spatiotemporal window.
        :param x1: float
        :param y1: float
        :param x2: float
        :param y2: float
        :param t1: timestamp
        :param t2: timestamp
        :return: set of int
        """
        spatial_result = self.spatial_window_query(x1, y1, x2, y2)
        temporal_result = self.temporal_window_query(t1, t2)
        return spatial_result.intersection(temporal_result)


In [84]:
from shapely.geometry import Point
import time

# Create G-IT index instance and insert toy dataset
sf_xmin, sf_xmax, sf_ymin, sf_ymax = tdf.bounds.minx.min(), tdf.bounds.maxx.max(), tdf.bounds.miny.min(), tdf.bounds.maxy.max()
git_index = GIT(sf_xmin=sf_xmin, sf_xmax=sf_xmax, sf_ymin=sf_ymin, sf_ymax=sf_ymax, delta_x=10, delta_y=10)

# Insert toy dataset
tdf['g_pair'] = list(zip(tdf.ts, tdf.geom))
cc = 0
tdf_minimized = tdf
for x in tdf_minimized.id.unique():
    tic = time.time()
    traj = Trajectory(x, tdf_minimized[tdf_minimized.id == x].sort_values(ascending=True,by='ts').g_pair.to_list())
    git_index.insert(traj)
    cc += tdf_minimized[tdf_minimized.id == x].shape[0]
    tac = time.time()
    print(f"processed - {(cc / tdf_minimized.shape[0]) * 100}% - {tac - tic} seconds - {tdf_minimized[tdf_minimized.id == x].shape[0]} trajectories")

processed - 0.006221017188152073% - 17.825547218322754 seconds - 24 trajectories
processed - 0.055989154693368653% - 196.53876185417175 seconds - 192 trajectories
processed - 0.062210171881520726% - 40.831552028656006 seconds - 24 trajectories
processed - 0.08087322344597694% - 108.08674573898315 seconds - 72 trajectories
processed - 0.10575729219858523% - 102.0936381816864 seconds - 96 trajectories
processed - 0.11197830938673731% - 25.215133666992188 seconds - 24 trajectories
processed - 0.342155945348364% - 1150.9454746246338 seconds - 888 trajectories
processed - 0.3685952683980103% - 159.2501564025879 seconds - 102 trajectories
processed - 0.3748162855861624% - 33.289790630340576 seconds - 24 trajectories
processed - 0.44013696606175917% - 383.40909814834595 seconds - 252 trajectories
processed - 0.4712420520025195% - 181.65364456176758 seconds - 120 trajectories
processed - 0.4774630691906716% - 26.032546520233154 seconds - 24 trajectories
processed - 0.48990510356697575% - 33.54

In [ ]:
# function to output two randomly ordered timestamps from ts column in dataframe tdf
def get_random_timestamps(tdf):
    ts = tdf.ts.unique()
    ts1 = np.random.choice(ts)
    ts2 = np.random.choice(ts)
    while ts1 == ts2:
        ts2 = np.random.choice(ts)
    return ts1, ts2


#function to generate two random coordinates (i.e., (x1, y1), (x2, y2)) that will representa bounding box (envelope) from GIT.grid
def get_random_bbox(git_index):
    x1 = np.random.uniform(git_index.sf_xmin, git_index.sf_xmax)
    x2 = np.random.uniform(git_index.sf_xmin, git_index.sf_xmax)
    y1 = np.random.uniform(git_index.sf_ymin, git_index.sf_ymax)
    y2 = np.random.uniform(git_index.sf_ymin, git_index.sf_ymax)
    return x1, y1, x2, y2
    

In [ ]:
# Test cases for each query
# Spatial window query
x1, y1, x2, y2 = get_random_bbox(git_index)
print("Spatial window query:")
print(git_index.spatial_window_query(x1, y1, x2, y2))

# Temporal window query
ts1,ts2 = get_random_timestamps(tdf_minimized)
print("Temporal window query:")
print(git_index.temporal_window_query(ts1,ts2))


# Spatiotemporal window query
# ts1,ts2 = get_random_timestamps(tdf_minimized)
# x1, y1, x2, y2 = get_random_bbox(git_index)
print("Spatiotemporal window query:")
print(git_index.spatiotemporal_window_query(x1, y1, x2, y2, ts1,ts2))

Spatial window query:
{2410, 2892}
Temporal window query:
{1562, 2898, 2892, 3741}
Spatiotemporal window query:
{2892}
